Definitions of Classes and Methods

Needs Troubleshooting

In [1]:
# Progress Counter
import sys

class progressCounter():

  def __init__(self, num_iterations, name=''):
    self.progress = 0
    self.N = num_iterations
    self.name = name

  def check_pt(self, i):
    curr_progress = int(i/(self.N-1) * 100)
    if curr_progress - self.progress > 0:
      self.progress = curr_progress
      sys.stdout.write("\r{1} Progress: {0}%".format(self.progress, self.name))
      sys.stdout.flush()

In [2]:
# Cosine Similarity Function
import numpy as np
from numpy import linalg as la
# takes in 2 numpy arrays
def compute_cosine_similarity(v1, v2):
  v1 = v1/la.norm(v1) # convert to unit vector
  v2 = v2/la.norm(v2)
  return np.dot(v1,v2) # compute dot product

In [3]:
# Text Cleaning & Preprocessing
import nltk
from nltk import word_tokenize
import string
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
# Cleans text of punctuation and stopwords
def pre_process(raw_text):
    raw_text = raw_text.lower()
    text_list = word_tokenize(raw_text)
    text_list = [word.lower() for word in text_list if word.lower() not in stopwords.words('english') and word not in string.punctuation]
    return text_list


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
# Building Dictionaries
def addAllToDict(raw_doc, curr_dict):
  text_list = pre_process(raw_doc)
  for word in text_list:
    # add to dictionary
    val = curr_dict.setdefault(word,0)
    curr_dict[word] = val+1
  return len(text_list)

In [5]:
# Inputs are Dictionaries
def createOperandsFromDict(d1,n1,d2,n2):
  d1 = d1.copy()
  d2 = d2.copy() 
  for e in d1:
    d2.setdefault(e,0) # if not in doc_cmp, add word set value to 0
  for e in d2:
    d1.setdefault(e,0) # if e not in vocab_cmp, add word and set val to 0
  # create final comparison vectors
  d1 = np.array([e[1] for e in sorted(d1.items())], dtype=np.double)/n1
  d2 = np.array([e[1] for e in sorted(d2.items())], dtype=np.double)/n2
  return (d1,d2)

Begin Script

In [6]:
# Import Data Set
from sklearn.datasets import fetch_20newsgroups

newsgroups_train = fetch_20newsgroups(data_home=None, subset='train', categories=None, shuffle=True, random_state=42, remove=(), download_if_missing=True, return_X_y=False)
newsgroups_test = fetch_20newsgroups(data_home=None, subset='test', categories=None, shuffle=True, random_state=42, remove=(), download_if_missing=True, return_X_y=False)



Train Data:

In [7]:
# Set Train Variables
data = newsgroups_train.data
indices = newsgroups_train.target
categories = newsgroups_train.target_names

Build the Collective Vocabulary for each category:

In [8]:
# Building Dictionaries
data_len = len(indices)
total_words = [0]*20

pC1 = progressCounter(data_len, 'Training')

# instantiate empty dicts
vocab_dict = [0]*20
for i in range(20):
  vocab_dict[i] = dict()

# create the dictionaries
for i in range(data_len):
  index = indices[i] # select category 
  num_words = addAllToDict(data[i], vocab_dict[index])
  total_words[index] += num_words

  pC1.check_pt(i)

Training Progress: 100%

Test Data:

In [9]:
data = newsgroups_test.data
pred_true = newsgroups_test.target
categories = newsgroups_test.target_names

In [ ]:
# Building Doc Vector (Test Data)

hits = 0
misses = 0
total_data_len = len(pred_true)
pc2 = progressCounter(total_data_len, 'Testing')
for i in range(total_data_len):  #outer layer cover all examples in data_len

  # create doc dictionary
  doc_dict = dict()
  num_words = addAllToDict(data[i],doc_dict)

  # Comparisons Begin
  prob = [0]*20
  for j in range(20): #loop through categories

    (doc_cmp, vocab_cmp) = createOperandsFromDict(doc_dict,num_words,vocab_dict[j],total_words[j])

    prob[j] = compute_cosine_similarity(doc_cmp, vocab_cmp)

  #endfor
  val = max(prob)
  pred = prob.index(val)

  if pred == pred_true[i]:
    hits+=1
  else:
    misses+=1

  pc2.check_pt(i)
#endfor
acc = int(hits/(hits+misses)*100)
print("Accuracy = {0}%".format(acc))


Testing Progress: 2%